In [7]:
from pathlib import Path

import cv2, pickle
import numpy as np

from dex_retargeting.constants import RobotName, RetargetingType, HandType, get_default_config_path
from dex_retargeting.retargeting_config import RetargetingConfig
from dex_retargeting.seq_retarget import SeqRetargeting
from single_hand_detector import SingleHandDetector

In [8]:
robot_name = RobotName.ability
retargeting_type = RetargetingType.dexpilot
hand_type = HandType.right

config_path = get_default_config_path(robot_name, retargeting_type, hand_type)
robot_dir = Path().absolute().parent / "ability_hand" / "assets" 
RetargetingConfig.set_default_urdf_dir(str(robot_dir))
retargeting = RetargetingConfig.load_from_file(config_path).build()

video_path = "data/human_hand_video.mp4"
output_path = "data/ability_joints.pkl"

 Mimic joint adaptor enabled. The mimic joint tags in the URDF will be considered during retargeting.
To disable mimic joint adaptor, consider setting ignore_mimic_joint=True in the configuration. 


In [9]:
cap = cv2.VideoCapture(video_path)

detector = SingleHandDetector(hand_type="Right", selfie=False)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

data = []

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    rgb = frame[..., ::-1]
    num_box, joint_pos, keypoint_2d, mediapipe_wrist_rot = detector.detect(rgb)
    
    retargeting_type = retargeting.optimizer.retargeting_type
    indices = retargeting.optimizer.target_link_human_indices
    if retargeting_type == "POSITION":
        indices = indices
        ref_value = joint_pos[indices, :]
    else:
        origin_indices = indices[0, :]
        task_indices = indices[1, :]
        ref_value = joint_pos[task_indices, :] - joint_pos[origin_indices, :]
    qpos = retargeting.retarget(ref_value)
    data.append(qpos)

meta_data = dict(
    config_path=config_path,
    dof=len(retargeting.optimizer.robot.dof_joint_names),
    joint_names=retargeting.optimizer.robot.dof_joint_names,
)

output_path = Path(output_path)
output_path.parent.mkdir(parents=True, exist_ok=True)
with output_path.open("wb") as f:
    pickle.dump(dict(data=data, meta_data=meta_data), f)

retargeting.verbose()
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1722970909.668605  448715 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2 Pro
W0000 00:00:1722970909.674650  450524 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1722970909.680156  450524 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/sy/miniforge3/envs/hand_diffusion/lib/python3.9/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Retargeting 621 times takes: 1.070062832000076s
Last distance: 0.03218003782018519


In [11]:
import sys
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,get_idxs,np_uv

xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)
env.set_geom_color(
    rgba                = (0.2,0.2,0.2,0.9),
    body_names_to_color = [
        'robot1/end_link','robot1/index_L1','robot1/index_L2','robot1/index_tip',
        'robot1/middle_L1','robot1/middle_L2','robot1/middle_tip',
        'robot1/ring_L1','robot1/ring_L2','robot1/ring_tip',
        'robot1/pinky_L1','robot1/pinky_L2','robot1/pinky_tip',
        'robot1/thumb_L1','robot1/thumb_L2','robot1/thumb_tip',
        'robot2/end_link','robot2/index_L1','robot2/index_L2','robot2/index_tip',
        'robot2/middle_L1','robot2/middle_L2','robot2/middle_tip',
        'robot2/ring_L1','robot2/ring_L2','robot2/ring_tip',
        'robot2/pinky_L1','robot2/pinky_L2','robot2/pinky_tip',
        'robot2/thumb_L1','robot2/thumb_L2','robot2/thumb_tip'],
)

right_hand_joint_names = ['robot2/' + i for i in meta_data['joint_names']]
right_hand_idxs = get_idxs(env.joint_names,right_hand_joint_names)
right_hand_joint_data = np.array(data)

In [12]:
env.init_viewer(viewer_title='Dual Arms',viewer_width=800,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

q_init = np.zeros(env.model.nq)
q_init[21] = -np.pi/2

env.forward(q=q_init)
tick = 0
seq_len = right_hand_joint_data.shape[0]

while(env.is_viewer_alive() and tick < seq_len):
    env.forward(q=right_hand_joint_data[tick],joint_idxs=right_hand_idxs)
    env.render()
    env.plot_T(p=np.zeros(3),R=np.eye(3),label='Tick: %d/%d'%(tick,seq_len),PLOT_AXIS=False)
    
    tick += 1
    if tick == seq_len:tick = 0

env.close_viewer()

Pressed ESC
Quitting.
